## 题目：寻找关键用户和分析社交关系

### data.json 数据描述
#### 1. 用户节点数据包含用户ID和他们的"影响力分数"，格式为["用户名称", 影响力分数]。
#### 2. 边关系数据包含每两个用户之间的连接强度（即边的权重），格式为["用户名称1", "用户名称2", 关系权重]。
例如：
- 节点：["Yanyan-Wang", 50.95], ["Aarebecca", 23.82], ...
- 边：["Yanyan-Wang", "Aarebecca", 25.3], ["Yanyan-Wang", "pomelo-nwu", 25.09], ...

### 问题
#### 1. 社交网络构建与分析
- a. 使用邻接矩阵或者字典表示法来表示这个社交关系网络。
- b. 请找出每个用户的直接朋友（即与其存在边关系的节点），并用列表形式列出。

#### 2. 统计与度中心性分析
- a. 计算每个用户的度数（即直接朋友数量），列出度数最多的前5个用户及其朋友数量。
- b. 通过影响力分数（如上所示），找出影响力最高的前5个用户。
- c. 综合考虑影响力分数和朋友数量，列出对社交网络最有影响力的用户（可以选择将两者按一定比例加权，例如影响力得分的50%和朋友数量的50%）。

#### 3. 推荐新朋友
- a. 对于用户 Yanyan-Wang，请找出其“朋友的朋友”中最有可能成为新朋友的用户，并列出推荐的理由（例如有最多共同的朋友）。
- b. 考虑用户之间的连接强度，找出与 Yanyan-Wang 连接强度最高的5个用户（无论是否已经是直接朋友），并列出推荐排序。
 
#### 4. 网络图的可视化
- a. 使用图论可视化工具绘制用户关系网络图，展示关键用户和他们的连接情况。
- b. 你可以对图中的节点按“影响力得分”进行着色，节点的大小则代表度数，尝试解释在图中哪些用户在社交网络中最具影响力。

In [ ]:
import json
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt

file_path = "C:/Users/Chlor/Downloads/hw4_data.json"
with open(file_path,'r')as f:
    data = json.load(f)
####任务一
#节点与边
nodes = data['nodes']
edges = data['edges']

#用邻接字典表示
adjacency_dict = defaultdict(list)

for edge in edges:
    user1, user2, weight = edge
    adjacency_dict[user1].append((user2, weight))
    adjacency_dict[user2].append((user1, weight)) 

#找出每个用户的直接朋友
direct_friends = {user: [friend[0] for friend in friends] for user, friends in adjacency_dict.items()}

print("邻接字典:")
for user, friends in adjacency_dict.items():
    print(f"{user}: {friends}")

print("\n直接朋友列表:")
for user, friends in direct_friends.items():
    print(f"{user}: {friends}")

###任务二
#计算每个用户的直接朋友数量
degree_centrality = {user: len(friends) for user, friends in direct_friends.items()}
#前五
top5_degree_users = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

print("度数最多的前5个用户及其朋友数量:")
for user, degree in top5_degree_users:
    print(f"{user}: {degree}")

#找出影响力最高的前5个用户
influence_scores = {user: score for user, score in nodes}
top5_influence_users = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)[:5]

print("\n影响力分数最高的前5个用户:")
for user, score in top5_influence_users:
    print(f"{user}: {score}")

#综合考虑影响力分数和朋友数量，计算综合得分
weighted_scores = {
    user: 0.5 * influence_scores.get(user, 0) + 0.5 * degree_centrality.get(user, 0)
    for user in influence_scores
}

most_influential_user = max(weighted_scores.items(), key=lambda x: x[1])

print("\n综合影响力得分最高的用户:")
print(f"{most_influential_user[0]}: {most_influential_user[1]}")

###任务三
# 找出 Yanyan-Wang 的“朋友的朋友”中最有可能成为新朋友的用户
def recommend_friends(user, adjacency_dict):
    user_friends = set(direct_friends[user]) 
    friends_of_friends = defaultdict(int) 

    # 统计共同朋友数
    for friend in user_friends:
        for fof, _ in adjacency_dict[friend]: 
            if fof != user and fof not in user_friends:
                friends_of_friends[fof] += 1

    recommended_friend = max(friends_of_friends.items(), key=lambda x: x[1], default=(None, 0))
    return recommended_friend

#推荐 Yanyan-Wang 的朋友的朋友
recommended_friend, mutual_friends = recommend_friends("Yanyan-Wang", adjacency_dict)

print("\n推荐的新朋友及理由:")
if recommended_friend:
    print(f"推荐新朋友: {recommended_friend}, 共同朋友数: {mutual_friends}")
else:
    print("没有可以推荐的新朋友")

#找出与 Yanyan-Wang 连接强度最高的5个用户
def strongest_connections(user, edges):
    connection_strengths = [
        (user1 if user2 == user else user2, weight)
        for user1, user2, weight in edges
        if user1 == user or user2 == user
    ]
    return sorted(connection_strengths, key=lambda x: x[1], reverse=True)[:5]

top5_strongest = strongest_connections("Yanyan-Wang", edges)

print("\n与 Yanyan-Wang 连接强度最高的5个用户:")
for user, strength in top5_strongest:
    print(f"{user}: {strength}")

###任务四
G = nx.Graph()
# 添加节点到图中，节点大小根据度数，颜色根据影响力得分
for user, score in nodes:
    degree = degree_centrality.get(user, 0)
    G.add_node(user, size=degree, influence=score, label=user)

# 添加边到图中，边的权重代表连接强度
for edge in edges:
    user1, user2, weight = edge
    G.add_edge(user1, user2, weight=weight)

# 设置节点属性：大小和颜色
node_sizes = [data['size'] * 50 for _, data in G.nodes(data=True)]  # 节点大小与度数成比例
node_colors = [data['influence'] for _, data in G.nodes(data=True)]  # 节点颜色由影响力得分决定

# 绘制网络图
plt.figure(figsize=(12, 8))

pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_edges(G, pos, alpha=0.5)
nodes = nx.draw_networkx_nodes(
    G,
    pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap=plt.cm.viridis,
    alpha=0.8
)
nx.draw_networkx_labels(G, pos, font_size=8)

plt.colorbar(nodes, label="影响力得分")

# 添加标题
plt.title("用户关系网络图", fontsize=14)
plt.axis("off")
plt.show()